In [1]:
import scanpy as sc, matplotlib.pyplot as plt, seaborn as sns, numpy as np, pandas as pd
sc.settings.set_figure_params(dpi=80, facecolor="white", frameon=False)
from tqdm.auto import tqdm
sc.settings.verbosity = 3

In [2]:
%%time

adata = sc.read_h5ad("Kidney_Combined_cellbender_v2.h5ad")
columns_to_drop = [col for col in adata.obs.columns if "RNA_snn" in col or "_prob" in col]
adata.obs.drop(columns=columns_to_drop, inplace=True)
adata

CPU times: user 4.78 s, sys: 8.58 s, total: 13.4 s
Wall time: 23.6 s


AnnData object with n_obs × n_vars = 2340396 × 36601
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name'

In [3]:
sc.pp.filter_cells(adata, min_genes=1)
sc.pp.filter_genes(adata, min_counts=1)

filtered out 364830 cells that have less than 1 genes expressed
filtered out 2464 genes that are detected in less than 1 counts


In [4]:
# appears to be a gene/transcript identifier, specifically an NCBI GenBank accession number
adata.var

,n_counts
MIR1302-2HG,350.0
FAM138A,2.0
OR4F5,19.0
AL627309.1,17495.0
AL627309.2,118.0
...,...
AC141272.1,400.0
AC023491.2,101.0
AC007325.1,101.0
AC007325.4,25613.0


In [5]:
adata

AnnData object with n_obs × n_vars = 1975566 × 34137
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes'
    var: 'n_counts'

In [6]:
%%time

adata = sc.read_h5ad("Kidney_Combined_hvg5k.h5ad")
columns_to_drop = [col for col in adata.obs.columns if "RNA_snn" in col or "_prob" in col]
adata.obs.drop(columns=columns_to_drop, inplace=True)
adata

CPU times: user 44.3 s, sys: 2.39 s, total: 46.7 s
Wall time: 47 s


AnnData object with n_obs × n_vars = 1975561 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined'
    var: 'n_counts', 'highly_variable', 'highly_variable_rank', 'me

In [12]:
adata.obsm

AxisArrays with keys: Harmony, Scanorama, X_bbknn, X_pca, X_scANVI, X_scVI, X_tsne, X_umap, scPoli

In [19]:
snrna_dataset = [
    'Kidney_Muto2', 
    'Kidney_Muto',
    'Kidney_Wilson2',
    'Kidney_Wilson',
    'Kidney_Lake', 
    'Kidney_Wu'
]

for item in snrna_dataset:
    assert item in adata.obs["Project_ID"].values


adata.obs["modality"] = "scrna"
mask = [item in snrna_dataset for item in adata.obs["Project_ID"].values]
adata.obs.loc[mask, "modality"] = "snrna"

In [20]:
%%time
adata.write_h5ad("Kidney_Combined_hvg5k.h5ad", compression="gzip")

CPU times: user 4min 10s, sys: 2.03 s, total: 4min 12s
Wall time: 4min 15s


In [21]:
selected_dataset = [
    'Kidney_BenPublished', 
    'Kidney_BenUnpublished',
    'Kidney_Krebs',
    'Kidney_Liao',
    'Kidney_Malone',
    'Kidney_Muto', 
    'Kidney_Raji', 
    'Kidney_Wilson',
    'Kidney_Krishna',
    'Kidney_Wu'
]

adata_labeled = adata[adata.obs["harmonized_celltype"] != 'nan']
adata_labeled = adata_labeled[[_batch in selected_dataset for _batch in adata_labeled.obs["Project_ID"]]]
adata_labeled

View of AnnData object with n_obs × n_vars = 494637 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_

In [22]:
adata_labeled

View of AnnData object with n_obs × n_vars = 494637 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_

In [23]:
# https://docs.google.com/spreadsheets/d/1yS3yNlVnIlsGclgao0R9EeTPQTKF8Nx4wPupjzM3mrg/edit?gid=0#gid=0

cell_types = [
    "SULF1+ EC-AEA", "EC-AEA", "Macro", "Macro", "cycMacro", "CCD-IC-A",
    "OMCD-IC-A", "CNT-PC", "CNT-PC", "DTL", "DTL", "DTL", "EC-DVR",
    "SLC6A6+ EC-AEA", "EC-GC", "EC-PTC", "EC-PTC", "EC-AVR", "IC-B",
    "VSMC", "Pericyte", "MAST", "MC", "MD", "cycNK/T", "cycNK/T", "DC1",
    "DC2", "IC-B|CNT doub", "POD", "POD", "PEC", "PT-S1/2", "PT-S1/2_nuc",
    "PT-S3", "dPT", "dPT", "TAL", "TAL", "TAL", "ATL", "Treg", "CD4 T",
    "CD4 T", "CD4 T", "Th17", "CD8 T", "CD8 T", "CD8 T", "ILC3", "NKT",
    "Ciliated", "DCT", "EC-LYM", "OMCD-PC", "IMCD-PC", "MFAP5+aFIB",
    "aFIB", "aFIB", "aFIB", "aPT", "aPT", "CCD-PC", "CCD-PC", "DCT",
    "Neuron", "RBC", "B", "B", "PL", "TAL", "VWF+ EC-AVR", "VWF+ EC-AVR",
    "cycEC", "cycPT", "CD16+ NK", "CD56bright NK", "pDC", "cMono", "ncMono",
    "cycPapE", "PapE", "γδT", "dC-IC-A"
]


# cell_types = [ct.lower() for ct in cell_types]

len(np.unique(cell_types))

62

In [24]:
for celltype in adata_labeled.obs["harmonized_celltype"].unique():
    harmomized_celltype = celltype.replace("LowQ ", "").replace(" doub", "").replace("HighM ", "")
#     harmomized_celltype = celltype
    if harmomized_celltype not in np.unique(cell_types):
        if "|" not in celltype:
            count = (adata_labeled.obs["harmonized_celltype"] == celltype).sum()
            print("CellType: %s; Count: %d" % (celltype, count))

CellType: LowQ PT; Count: 7978
CellType: HighM PT; Count: 7793
CellType: LowQ CNT/DCT; Count: 69
CellType: LowQ S100A6+ EC-PTC; Count: 392
CellType: LowQ EC-DCR; Count: 548
CellType: LowQ DC/Macro; Count: 19
CellType: LowQ NK; Count: 327
CellType: LowQ dOMCD-PC; Count: 2668
CellType: LowQ EC; Count: 2653
CellType: MFAP5+ aFIB; Count: 282
CellType: EC-VSMC doub; Count: 1146
CellType: GSM4339777-specific VWF+ EC-AVR; Count: 55
CellType: Macro-VSMC doub; Count: 392
CellType: DCT-CNT doub; Count: 423
CellType: HighM epithelial; Count: 274
CellType: LowQ ATL/DTL; Count: 1158
CellType: Epi-endo doub; Count: 51
CellType: LowQ-Macro; Count: 465
CellType: Epi-VSMC doub; Count: 243
CellType: LowQ neuron; Count: 298
CellType: S100A6+ aPT; Count: 1190
CellType: Endo-VSMC doub; Count: 438
CellType: S100A4+ MYRIP+ EC-DCR; Count: 95
CellType: LowQ PCDH15+aPT; Count: 54
CellType: PAC; Count: 43


In [25]:
for celltype in adata_labeled.obs["harmonized_celltype"].unique():
    harmomized_celltype = celltype
    if harmomized_celltype not in np.unique(cell_types):
        count = (adata_labeled.obs["harmonized_celltype"] == celltype).sum()
        print("CellType: %s; Count: %d" % (celltype, count))

CellType: LowQ PT; Count: 7978
CellType: HighM PT; Count: 7793
CellType: LowQ TAL; Count: 2150
CellType: LowQ CNT/DCT; Count: 69
CellType: LowQ S100A6+ EC-PTC; Count: 392
CellType: PT|S100A6- CD4 T doub; Count: 43
CellType: LowQ Macro; Count: 5090
CellType: CNT|CCD-IC-A doub; Count: 38
CellType: LowQ EC-DCR; Count: 548
CellType: PT|CCD-IC-A doub; Count: 30
CellType: HighM cMono; Count: 17
CellType: Macro|aPT doub; Count: 1793
CellType: Stressed Macro|aPT doub; Count: 255
CellType: LowQ EC-PTC; Count: 70
CellType: LowQ DC/Macro; Count: 19
CellType: LowQ NK; Count: 327
CellType: PEC|TAL doub; Count: 41
CellType: Macro|CD8 T doub; Count: 8
CellType: LowQ CD4 T; Count: 15
CellType: DCT|aPT doub; Count: 6
CellType: dPT|T doub; Count: 2788
CellType: LowQ dPT; Count: 27
CellType: TAL|PT doub; Count: 43
CellType: CCD-IC-A|PT doub; Count: 779
CellType: TAL|EC doub; Count: 3664
CellType: LowQ dOMCD-PC; Count: 2668
CellType: CNT|Macro doub; Count: 363
CellType: aPT|aFIB doub; Count: 397
CellType:

In [29]:
# np.sum([celltype.lower() in cell_types for celltype in adata.obs["harmonized_celltype"]])

411838

In [26]:
np.sum([celltype in cell_types for celltype in adata_labeled.obs["harmonized_celltype"]])

411838

In [27]:
adata_harmonized = adata_labeled[[celltype in cell_types for celltype in adata_labeled.obs["harmonized_celltype"]]]
adata_harmonized

View of AnnData object with n_obs × n_vars = 411838 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_

In [28]:
adata_harmonized.write_h5ad("Kidney_Combined_hvg5k_LabeledSubset.h5ad", compression="gzip")